In [262]:
import numpy as np
import pandas as pd
import math

In [263]:
data = pd.read_csv("id3.csv")
data

,Outlook,Temperature,Humidity,Wind,Answer
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes
5,rain,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rain,mild,normal,weak,yes


In [264]:
input = data.drop('Answer',axis=1)
input.head(3)

,Outlook,Temperature,Humidity,Wind
0,sunny,hot,high,weak
1,sunny,hot,high,strong
2,overcast,hot,high,weak


In [265]:
target = data['Answer']
target.head(3)

0     no
1     no
2    yes
Name: Answer, dtype: object

In [266]:
def attr_list(df):
    attributes = df.columns.tolist()[:-1]
    return attributes
print(attr_list(data))

['Outlook', 'Temperature', 'Humidity', 'Wind']


In [267]:
def attr_values(attributes,data):
    values = {}
    for i in attributes:
        values[i] = data[i].unique()
    return values
print(attr_values(attr_list(data),data))

{'Outlook': array(['sunny', 'overcast', 'rain'], dtype=object), 'Temperature': array(['hot', 'mild', 'cool'], dtype=object), 'Humidity': array(['high', 'normal'], dtype=object), 'Wind': array(['weak', 'strong'], dtype=object)}


In [268]:
def entropy(data_n):
    p=len(data_n.where(data_n['Answer']=='yes').dropna())
    n=len(data_n.where(data_n['Answer']=='no').dropna())
    if(n==p):
        return 1
    if(n==0 or p==0):
        return 0
    e=(((-1)*p*math.log(p/(p+n),2)/(p+n)) + ((-1)*n*math.log(n/(p+n),2)/(p+n)))
    return float(e)

In [269]:
def information(attribute,att_values,data_n):
    p = len(data_n.where(data_n['Answer']=='yes').dropna())
    n = len(data_n.where(data_n['Answer']=='no').dropna())
    i=0
    for x in att_values:
        p_val = len(data_n.where((data_n[attribute]==x) & (data_n['Answer']=='yes')).dropna())
        n_val = len(data_n.where((data_n[attribute]==x) & (data_n['Answer']=='no')).dropna())
        i += ((p_val+n_val)/(p+n))*(entropy(data_n.where(data_n[attribute]==x).dropna()))
    return i

In [270]:
class Node:
    def __init__(self,attribute):
        self.attribute=attribute
        self.children=[]
        self.answer = {}

In [271]:
NG = ['NG','NG','NG']
root = Node(NG)
print(root.attribute)

['NG', 'NG', 'NG']


In [272]:
def Compute_tree(df,root):
    if(len(set(df[df.columns.tolist()[-1]])) == 1):
        return set(df[df.columns.tolist()[-1]])
    e = entropy(df)
    attributes = attr_list(df)
    values = attr_values(attributes,df)
    gain_of_attr = {}
    gain = 0
    best_attr = attributes[0]
    for a in attributes:
        gain_of_attr[a] = e - information(a,values[a],df)
        gain,best_attr = (gain_of_attr[a],a) if(gain_of_attr[a] > gain) else (gain,best_attr)
    node = Node(best_attr)
    node.children = values[best_attr]
    for x in node.children:
        node.answer[x]=Compute_tree(df.where(df[best_attr]==x).dropna(),root)
    return node
r=Compute_tree(data,root)
    
        
    
        

In [273]:
#print(r.attribute,r.children,r.answer['sunny'].attribute)
print((" ")*(len(r.children[0])+5),r.attribute)
for c in r.children:
    if(r.answer[c]=={'yes'} or r.answer[c]=={'no'}):
        print(c,r.answer[c],end="   ")
    else:
        print(c,end="     ")

           Outlook
sunny     overcast {'yes'}   rain     

In [250]:
def Display_tree(node,lable):
    if(node=={'yes'} or node=={'no'}):
        return
    print((" ")*(24),"",str(node.attribute).upper(),"(",lable,")")
    print("Brnches of :",node.attribute,"\n")
    for c in node.children:
        if(node.answer[c]!={'yes'} and node.answer[c]!={'no'}):
            print(c,end="\n")
        else:
            print(c,"\n",node.answer[c],end="\n")
        print()
    for c in node.children:
        if(node.answer[c]!={'yes'} and node.answer[c]!={'no'}):
                Display_tree(node.answer[c],"child of "+str(node.attribute).upper()+" through the branch : "+c)  
    

In [251]:
Display_tree(r,"root")

                          OUTLOOK ( root )
Brnches of : Outlook 

sunny

overcast 
 {'yes'}

rain

                          HUMIDITY ( child of OUTLOOK through the branch : sunny )
Brnches of : Humidity 

high 
 {'no'}

normal 
 {'yes'}

                          WIND ( child of OUTLOOK through the branch : rain )
Brnches of : Wind 

weak 
 {'yes'}

strong 
 {'no'}

